In [4]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
import json

In [6]:
json.dumps(np.nan)

'NaN'

In [9]:
query_data = [{1:3, 2:4, 4:np.nan}]

In [12]:
[{k: 0 if type(v) == type(np.nan) else v for k, v in d.items()} for d in query_data]

[{1: 3, 2: 4, 4: 0}]

In [13]:
!ls

BenefitsCostSharing.csv
BenefitsCostSharing.zip
server.py
static
temp.json
templates
test.ipynb


In [14]:
dtypes = {
    'BenefitName': np.str, 'BusinessYear': np.uint16, 'CoinsOutofNet':np.str, 'EHBVarReason':np.str, 'Explanation':np.str, 'PlanId':np.str,
       'RowNumber':np.uint16, 'SourceName':np.str, 'StandardComponentId':np.str, 'StateCode':np.str,
       'StateCode2':np.str, 'VersionNum':np.uint16
}

In [15]:
# pd.read_csv??

In [16]:
ddata = dd.read_csv('BenefitsCostSharing.csv', dtype=dtypes, usecols=dtypes.keys())
ddata.reset_index(drop=True).compute()
# df = ddata.drop_duplicates()

,BenefitName,BusinessYear,CoinsOutofNet,EHBVarReason,Explanation,PlanId,RowNumber,SourceName,StandardComponentId,StateCode,StateCode2,VersionNum
0,Routine Dental Services (Adult),2014,20%,Above EHB,Combined annual benefit maximum of $1000 per y...,21989AK0010001-00,68,HIOS,21989AK0010001,AK,AK,6
1,Dental Check-Up for Children,2014,20%,Substantially Equal,See policy for additional limitations,21989AK0010001-00,104,HIOS,21989AK0010001,AK,AK,6
2,Basic Dental Care - Child,2014,40%,Substantially Equal,See policy for additional limitations,21989AK0010001-00,110,HIOS,21989AK0010001,AK,AK,6
3,Orthodontia - Child,2014,50%,Additional EHB Benefit,"24 month waiting period, See policy for additi...",21989AK0010001-00,111,HIOS,21989AK0010001,AK,AK,6
4,Major Dental Care - Child,2014,50%,Substantially Equal,See policy for additional limitations,21989AK0010001-00,112,HIOS,21989AK0010001,AK,AK,6
...,...,...,...,...,...,...,...,...,...,...,...,...
194339,Major Dental Care - Child,2016,50% Coinsurance after deductible,NaN,NaN,96480WV0110003-00,112,SERFF,96480WV0110003,WV,WV,2
194340,Basic Dental Care - Adult,2016,NaN,NaN,NaN,96480WV0110003-00,113,SERFF,96480WV0110003,WV,WV,2
194341,Orthodontia - Adult,2016,NaN,NaN,NaN,96480WV0110003-00,114,SERFF,96480WV0110003,WV,WV,2
194342,Major Dental Care - Adult,2016,NaN,NaN,NaN,96480WV0110003-00,115,SERFF,96480WV0110003,WV,WV,2


In [82]:
ddata

,BenefitName,BusinessYear,CoinsOutofNet,EHBVarReason,Explanation,PlanId,RowNumber,SourceName,StandardComponentId,StateCode,StateCode2,VersionNum
npartitions=21,,,,,,,,,,,,
,object,uint16,object,object,object,object,uint16,object,object,object,object,uint16
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [86]:
ddata.to_json??

In [118]:
ddata.size.compute()

60580896

In [101]:
ddata.columns

Index(['BenefitName', 'BusinessYear', 'CoinsOutofNet', 'EHBVarReason',
       'Explanation', 'PlanId', 'RowNumber', 'SourceName',
       'StandardComponentId', 'StateCode', 'StateCode2', 'VersionNum'],
      dtype='object')

In [17]:
ddata.query("(BenefitName.str.contains('Rou*ti', regex= True))").loc[:10, :].size.compute()

240

In [108]:
filters = {"BenefitName": "Rou*ti", "VersionNum": "<6"}

In [116]:
def build_query(filters):
    num_keys = ["BusinessYear", "RowNumber", "VersionNum"]
    query = []
    for k,v in filters.items():
        if ("<" in v or ">" in v) and k in num_keys:
            query.append(f"({k}{v})")
        elif "==" in v or "!=" in v:
            query.append(f"({k}{v})")
        else:
            
            query.append(f"({k}.str.contains('{v}', regex= True))")
    return " and ".join(query)


In [117]:
print(build_query({}))